# Dependencies and necessary imports

In [ ]:
!pip install mediapipe opencv-python numpy

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import time
import glob
mp_drawing = mp.solutions.drawing_utils  # Drawing utility
mp_pose = mp.solutions.pose  # Provides Pose estimation model

# Calculate angles

In [ ]:
def calculate_angle(a,b,c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
#   angle = np.arccos(np.dot(a-b, (c-b).T)/np.sqrt(np.dot(a-b, (a-b).T)*np.dot(c-b, (c-b).T)))
    angle = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(angle*180.0/np.pi)
    if angle > 180.0:
        angle = 360 - angle
    return angle

# Pose Detection algorithm

In [ ]:
cap = cv2.VideoCapture("KneeBend.mp4")

activate_video_record = 0

image_width = 640
image_height = 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, image_width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, image_height)

counter = 0
status = "down"
exercise_status = ""
color_tuple = None
img_array = []
original_image = None

# Setting up the mediapipe instance

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.2, enable_segmentation=True) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RG
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Storing the original image
        original_image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Nullify all pixel values brighter than 180, for boundary accentuation
        image = np.multiply(image, image < 180)

        # Processing the image and making detections
        results = pose.process(image)
        
        # Recolor image back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # try and except block for a scenario where no landmarks is obtained from an image 
        try:
            # Extract Landmarks
            landmarks = results.pose_landmarks.landmark


            # Landmarks for left_hip, left_knee, left_ankle
            left_hip = mp_pose.PoseLandmark.LEFT_HIP.value
            left_knee = mp_pose.PoseLandmark.LEFT_KNEE.value
            left_ankle = mp_pose.PoseLandmark.LEFT_ANKLE.value

            left_hip = [landmarks[left_hip].x , landmarks[left_hip].y]
            left_knee = [landmarks[left_knee].x, landmarks[left_knee].y, landmarks[left_knee].z]
            left_ankle = [landmarks[left_ankle].x, landmarks[left_ankle].y]

            # Landmarks for right_hip, right_knee, right_ankle
            right_hip = mp_pose.PoseLandmark.RIGHT_HIP.value
            right_knee = mp_pose.PoseLandmark.RIGHT_KNEE.value
            right_ankle = mp_pose.PoseLandmark.RIGHT_ANKLE.value


            right_hip = [landmarks[right_hip].x , landmarks[right_hip].y]
            right_knee = [landmarks[right_knee].x, landmarks[right_knee].y, landmarks[right_knee].z]
            right_ankle = [landmarks[right_ankle].x, landmarks[right_ankle].y]

            
            # Conditional assignment to determine which leg is closer to the camera
            if left_knee[2] - right_knee[2] > 0:
                hip = right_hip
                knee = right_knee
                ankle = right_ankle

            else:
                hip = left_hip
                knee = left_knee
                ankle = left_ankle

                
            # Calculate angle between hip, knee and the ankle
            angle = calculate_angle(hip, knee, ankle)

            # Counter increment, status, exercise_status display algorithm
            if angle > 170:
                if status == "up":
                    end = time.time()
                    if end - start > 8:
                        counter += 1
                        exercise_status = ""
                    else:
                        exercise_status = "Keep your knee bent"
                status = "down"
                
            if angle < 140 and status == "down":
                start = time.time()
                status = "up"
                exercise_status = ""

            # Assigning different colors as per the exercise status
            if exercise_status == "Keep your knee bent":
                color_tuple = (0,0,255)
            else:
                color_tuple = (0,255,0)
                
        except:
            pass

        # Assigning the image to the original image on which landmarks, counter and status is to be displayed
        image = original_image
        
        
        # Rendering landmarks, counter and status values on the image
        
        # Exercise status data
        cv2.rectangle(image, (image_width - image_width//3 ,0), (image_width, image_height//12), (250, 250, 250), -1)
        
        cv2.putText(image, exercise_status, (image_width - int(image_width/3.1), image_height//20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color_tuple, 1, cv2.LINE_AA)
                
            
        # Rep data
        cv2.rectangle(image, (0,0), (image_width//5, image_height//12), (255, 255, 255), -1)
        
        cv2.putText(image, 'REPS:', (image_width//70, image_height//36), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, str(counter), (image_width//8 , image_height//36), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
    
        
        # Status data
        cv2.putText(image, 'Status:', (image_width//70, image_height//14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        
        cv2.putText(image, status.upper(), (image_width//8 , image_height//14), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 2, cv2.LINE_AA)
    
        
        # Render the detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks,
                                  mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Show the output
        cv2.imshow('MediaPipe', image)
        if cv2.waitKey(10) == ord('q'):
            break

        # Write the image data on specified output file
        size = (image_width, image_height)
        if  activate_video_record == 0:
            out = cv2.VideoWriter('Project.avi',cv2.VideoWriter_fourcc(*'DIVX'), 20, size)
            activate_video_record = 1

        out.write(image)

    cap.release()
    cv2.destroyAllWindows()